In [5]:
#Qiskit Aqua: Build an algorithm
#Simple example of an algorithm
#Evolution Fidelity
#How to implement an algorithm and 
#Steps to configure and run the algorithm

import numpy as np
# Importing standard Qiskit libraries and configuring account
from qiskit import BasicAer, IBMQ
from qiskit.aqua.operators import MatrixOperator, op_converter
#from evolutionfidelity.evolutionfidelity import EvolutionFidelity
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua import QuantumInstance
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

Credentials are already in use. The existing account in the session will be replaced.


In [6]:
"""
The Fidelity of Quantum Evolution.
This is a simple tutorial example to show how to build an algorithm to extend
Qiskit Aqua library.
"""
import logging
import numpy as np
from qiskit import QuantumRegister
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import QuantumAlgorithm
from qiskit.aqua.operators import op_converter

logger = logging.getLogger(__name__)

class EvolutionFidelity(QuantumAlgorithm):

    CONFIGURATION = {
    }

    def __init__(self, operator, initial_state, expansion_order=1):
        self.validate(locals())
        super().__init__()
        self._operator = operator
        self._initial_state = initial_state
        self._expansion_order = expansion_order
        self._ret = {}

    """
    Once the algorithm has been initialized then run is called to carry out the computation
    and the result is returned as a dictionary.
    E.g., the `_run` method is required to be implemented for an algorithm.
    """

    def _run(self):
        evo_time = 1
        # get the groundtruth via simple matrix * vector
        state_out_exact = op_converter.to_matrix_operator(self._operator).evolve(
            self._initial_state.construct_circuit('vector'), evo_time, num_time_slices=1)

        qr = QuantumRegister(self._operator.num_qubits, name='q')
        circuit = self._initial_state.construct_circuit('circuit', qr)
        circuit += self._operator.evolve(
            None, evo_time, 1,
            quantum_registers=qr,
            expansion_mode='suzuki',
            expansion_order=self._expansion_order
        )
        result = self._quantum_instance.execute(circuit)
        state_out_dynamics = np.asarray(result.get_statevector(circuit))

        self._ret['score'] = state_fidelity(state_out_exact, state_out_dynamics)

        return self._ret

In [7]:
num_qubits = 2
temp = np.random.random((2 ** num_qubits, 2 ** num_qubits))
qubit_op = op_converter.to_weighted_pauli_operator(MatrixOperator(matrix=temp + temp.T))

initial_state = Zero(qubit_op.num_qubits)

algo = EvolutionFidelity(qubit_op, initial_state, expansion_order=1)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend)

result = algo.run(quantum_instance)
print(result['score'])

0.6560692464896263
